In [1]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
import requests

/home/tutorial1/Documents/rag-pinecone-serverless/lang/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
# Keys
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [4]:
# Init
pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                         environment=PINECONE_ENVIRONMENT)

embeddings = CohereEmbeddings(model="multilingual-22-12")
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [5]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)


/home/tutorial1/Documents/rag-pinecone-serverless/lang/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
chain.invoke("what is film noir?")

'Film noir is a cinematic term used primarily to describe stylish Hollywood crime dramas, particularly those that emphasize cynical attitudes and sexual motivations. The term is French for "black film" and was first applied by French critics to American films that emerged in the post-World War II era, which were marked by a dark, gloomy atmosphere and a sense of pessimism, fatalism, and menace. Classic film noir often features hard-boiled detectives, femme fatales, and complex narratives with a focus on the seedy underbelly of society. The visual style of film noir is characterized by high-contrast lighting, stark shadows, and unusual camera angles to enhance the sense of psychological tension and moral ambiguity.'

In [9]:
def fetch_wikipedia_page(id):
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"
    response = requests.get(url)
    data = response.json()
    page_content = list(data['query']['pages'].values())[0]['extract']
    return page_content

def fetch_url(x):
    urls = [doc.metadata['url'] for doc in x['context']]
    ids = [url.split('=')[-1] for url in urls]
    # First 32k tokens
    contents = [fetch_wikipedia_page(id)[:32000] for id in ids]    
    return {"context": contents, "question": x["question"]}

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | RunnableLambda(fetch_url)  
    | prompt
    | model
    | StrOutputParser()
)


In [10]:
chain.invoke("what is film noir?")

'Based on the context provided, which is empty, I cannot draw from any specific details you\'ve given. However, I can tell you that film noir is a genre of cinema characterized by its dark, moody atmosphere, crime and moral ambiguity, complex characters, and often a cynical outlook. The term itself is French for "black film" or "dark film." Film noir is associated with a visual style that includes low-key lighting, stark shadows, and compositions that create a sense of unease or disorientation. The genre\'s classic period is generally considered to be from the early 1940s to the late 1950s.'